In [1]:
import os
import pandas as pd
import numpy as np
import sklearn
import cv2
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

2021-11-12 01:57:37.915852: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


2021-11-12 01:57:46.754127: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-11-12 01:57:47.118089: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:0a:00.0 name: NVIDIA GeForce GTX 1080 computeCapability: 6.1
coreClock: 1.759GHz coreCount: 20 deviceMemorySize: 7.93GiB deviceMemoryBandwidth: 298.32GiB/s
2021-11-12 01:57:47.119480: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 1 with properties: 
pciBusID: 0000:0b:00.0 name: NVIDIA GeForce GTX 1080 computeCapability: 6.1
coreClock: 1.759GHz coreCount: 20 deviceMemorySize: 7.93GiB deviceMemoryBandwidth: 298.32GiB/s
2021-11-12 01:57:47.119537: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-11-12 01:57:47.175465: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcubl

In [3]:
os.chdir('/root/data/')

In [4]:
path = './OIDv4_ToolKit/OID/Dataset/train/'

In [5]:
rm -r ./OIDv4_ToolKit/OID/Dataset/train/Helmet/.ipynb_checkpoints

rm: cannot remove './OIDv4_ToolKit/OID/Dataset/train/Helmet/.ipynb_checkpoints': No such file or directory


In [6]:
img_np = np.array([])
label_np = np.array([])
for label in ['Bicycle helmet', 'Helmet', 'Human face']:
    for txt in os.listdir(f'{path}{label}/Label/'):
        img_np = np.append(img_np, np.array(f'{path}{label}/{txt[:-4]}.jpg'))
        label_np = np.append(label_np, np.array(label))

In [7]:
print(len(img_np))

22614


In [8]:
train_df = pd.DataFrame()
train_df['img_name'] = img_np
train_df['tag'] = label_np
train_df['label'] = label_np
train_df['label'] = train_df['label'].map({'Bicycle helmet' : 0, 'Helmet' : 0, 'Human face' : 1})

In [9]:
train_df.head()

,img_name,tag,label
0,./OIDv4_ToolKit/OID/Dataset/train/Bicycle helm...,Bicycle helmet,0
1,./OIDv4_ToolKit/OID/Dataset/train/Bicycle helm...,Bicycle helmet,0
2,./OIDv4_ToolKit/OID/Dataset/train/Bicycle helm...,Bicycle helmet,0
3,./OIDv4_ToolKit/OID/Dataset/train/Bicycle helm...,Bicycle helmet,0
4,./OIDv4_ToolKit/OID/Dataset/train/Bicycle helm...,Bicycle helmet,0


In [10]:
IMG_SIZE = 224
num_samples = train_df.shape[0]

In [11]:
train_df = sklearn.utils.shuffle(train_df)

In [12]:
label = train_df['label'].values.reshape(-1,1)

In [13]:
x = np.zeros(shape=(num_samples, IMG_SIZE, IMG_SIZE, 3 ), dtype="float32" )

In [14]:
index = 0
for img_path in tqdm(train_df['img_name']):
    try:
        img = cv2.imread(img_path).astype(np.float32)
        img = cv2.resize(img, (224,224))
        x[index] = img
        index += 1
    except:
        print(img_path)

100%|████████████████████████████████████████████████████████████| 22614/22614 [16:11<00:00, 23.27it/s]


In [15]:
x.shape

(22614, 224, 224, 3)

In [16]:
y = np.zeros(shape=(num_samples), dtype="float32" )

In [17]:
y = to_categorical(label)

In [18]:
y.shape

(22614, 2)

In [19]:
path = './OIDv4_ToolKit/OID/Dataset/validation/'

In [20]:
img_np = np.array([])
label_np = np.array([])
for label in ['Bicycle helmet', 'Helmet', 'Human face']:
    for txt in os.listdir(f'{path}{label}/Label/'):
        img_np = np.append(img_np, np.array(f'{path}{label}/{txt[:-4]}.jpg'))
        label_np = np.append(label_np, np.array(label))

In [21]:
validation_df = pd.DataFrame()
validation_df['img_name'] = img_np
validation_df['tag'] = label_np
validation_df['label'] = label_np
validation_df['label'] = validation_df['label'].map({'Bicycle helmet' : 0, 'Helmet' : 0, 'Human face' : 1})

In [22]:
IMG_SIZE = 224
num_samples = validation_df.shape[0]

In [23]:
validation_df = sklearn.utils.shuffle(validation_df)

In [24]:
val_label = validation_df['label'].values.reshape(-1,1)

In [25]:
val_y = to_categorical(val_label)

In [26]:
val_x = np.zeros(shape=(num_samples, IMG_SIZE, IMG_SIZE, 3 ), dtype="float32" )

In [27]:
index = 0
for img_path in tqdm(validation_df['img_name']):
    try:
        img = cv2.imread(img_path).astype(np.float32)
        img = cv2.resize(img, (224,224))
        val_x[index] = img
        index += 1
    except:
        print(img_path)

100%|██████████████████████████████████████████████████████████████| 3227/3227 [01:49<00:00, 29.38it/s]


In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [29]:
model = Sequential([
    Conv2D(32, kernel_size=(3,3), input_shape=(224, 224, 3), activation="relu", padding="SAME"),
    MaxPooling2D(pool_size=2, padding="SAME", strides=2),
    Conv2D(64, kernel_size=(3,3), activation="relu", padding="SAME"),
    MaxPooling2D(pool_size=2, padding="SAME", strides=2),
    Conv2D(128, kernel_size=(3,3), padding="SAME", activation="relu"),
    MaxPooling2D(pool_size=2, padding="SAME", strides=2),
    Flatten(),
    Dropout(0.5),
    Dense(128, activation="relu"),
    Dense(2, activation='softmax'),
])

2021-11-12 02:16:26.576444: I tensorflow/stream_executor/cuda/cuda_driver.cc:775] failed to allocate 7.04G (7558914304 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2021-11-12 02:16:26.576991: I tensorflow/stream_executor/cuda/cuda_driver.cc:775] failed to allocate 6.33G (6803022336 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2021-11-12 02:16:26.577782: I tensorflow/stream_executor/cuda/cuda_driver.cc:775] failed to allocate 5.70G (6122719744 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2021-11-12 02:16:26.578666: I tensorflow/stream_executor/cuda/cuda_driver.cc:775] failed to allocate 5.13G (5510447616 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2021-11-12 02:16:26.579535: I tensorflow/stream_executor/cuda/cuda_driver.cc:775] failed to allocate 4.62G (4959402496 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2021-11-12 02:16:26.580440: I tensorflow/stream_executor/cuda/cuda_driver.cc:775] failed to allocate 4.16

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0

In [31]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-6)
            ,loss='categorical_crossentropy'
            ,metrics=['acc']
            )

In [32]:
checkpoint_path = f"./model/tmp_checkpoint2.ckpt"
checkpoint = ModelCheckpoint(filepath=checkpoint_path, 
                                save_weights_only=True, 
                                save_best_only=True, 
                                monitor='val_loss', 
                                verbose=1)

In [33]:
model.fit(x,
          y,
          validation_data=(val_x, val_y),
          epochs=10,
          callbacks=[checkpoint],
         )

2021-11-12 02:16:27.475789: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 13616160768 exceeds 10% of free system memory.


Epoch 1/10


2021-11-12 02:16:33.994987: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2021-11-12 02:16:33.998558: E tensorflow/stream_executor/cuda/cuda_blas.cc:225] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-11-12 02:16:34.034063: E tensorflow/stream_executor/cuda/cuda_blas.cc:225] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-11-12 02:16:34.128133: E tensorflow/stream_executor/cuda/cuda_blas.cc:225] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-11-12 02:16:34.131751: E tensorflow/stream_executor/cuda/cuda_blas.cc:225] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-11-12 02:16:34.151928: E tensorflow/stream_executor/cuda/cuda_blas.cc:225] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-11-12 02:16:34.157830: E tensorflow/stream_executor/cuda/cuda_blas.cc:225] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021

ResourceExhaustedError:  OOM when allocating tensor with shape[32,32,224,224] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential/conv2d/Relu (defined at tmp/ipykernel_18220/3163090840.py:5) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_960]

Function call stack:
train_function


In [ ]:
test_path = './OIDv4_ToolKit/OID/Dataset/test/'

In [ ]:
img_np = np.array([])
label_np = np.array([])
for label in ['Bicycle helmet', 'Helmet', 'Human face']:
    for txt in os.listdir(f'{path}{label}/Label/'):
        img_np = np.append(img_np, np.array(f'{path}{label}/{txt[:-4]}.jpg'))
        label_np = np.append(label_np, np.array(label))

In [ ]:
test_df = pd.DataFrame()
test_df['img_name'] = img_np
test_df['tag'] = label_np
test_df['label'] = label_np
test_df['label'] = test_df['label'].map({'Bicycle helmet' : 0, 'Helmet' : 0, 'Human face' : 1})

In [ ]:
IMG_SIZE = 224
num_samples = test_df.shape[0]

In [ ]:
test_df.head()

In [ ]:
test_x = np.zeros(shape=(num_samples, IMG_SIZE, IMG_SIZE, 3 ), dtype="float32" )

In [ ]:
index = 0
for img_path in tqdm(test_df['img_name']):
    try:
        img = cv2.imread(img_path).astype(np.float32)
        img = cv2.resize(img, (224,224))
        test_x[index] = img
        index += 1
    except:
        print(img_path)

In [ ]:
test_label = test_df['label'].values.reshape(-1,1)

In [ ]:
test_y = to_categorical(test_label)

In [ ]:
model.evaluate(test_x, test_y)

In [ ]:
model.save("./model/demo2.h5")